In [17]:
%load_ext autoreload
%autoreload 2

import json
import sys
import re
import os
from pathlib import Path
from functools import partial
from itertools import chain

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay

import bhutils
from bhutils import bhutilspy as bh
from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from api.utils import parse_offset
from api.data.base import data_path, radar_path
from api.gui.base import View

In [17]:
curves_df = pd.DataFrame(
    [
        ('AUD', 'AUD.6M.STD', 'AUD.D.AUDC'),
        ('CAD', 'CAD.3M.STD', 'CAD.D.CADC'),
        ('CHF', 'CHF.6M.STD', 'CHF.D.CHFC'),
        ('EUR', 'EUR.6M.STD', 'EUR.D.EURC'),
        ('GBP', 'GBP.6M.STD', 'GBP.D.GBPC'),
        ('NOK', 'NOK.6M.STD', 'NOK.D.NOKC'),
        ('NZD', 'NZD.3M.STD', 'NZD.D.NZDC'),
        ('SEK', 'SEK.6M.STD', 'SEK.D.SEKC'),
        ('USD', 'USD.SOFR', 'USD.D'),
        ('JPY', 'JPY.6M.STD', 'JPY.D.JPYC'),
    ],
    columns=('ccy', 'std', 'disc')
).set_index('ccy')

In [1]:
class LiveCurvesLocal:
    def __init__(self, date=None, curves=None):
        self.date = date or pd.Timestamp.today().floor('d')
        self.curves = curves
        try:
            self.sheet_mkt = bh.bhCreateSheetMarket(
                _handle='sheet_mkt',
                _mkt='LIVE',
                _asof=self.date,
                _mktdefs=mkt_defs,
                _calprov=mx_cals,
                _mtgdts=mtg_dts,
                _crvdbsvr="SQLP1-A",
                _crvdbname="Curves",
                _mktdatasvr="localhost",
                _mktdataport=60000,
                _snapdbsvr="SQLP1-B",
                _snapdbname="Snap",
            )
        except TypeError:
            self.sheet_mkt = bh.bhCreateSheetMarket(
                _hdl='sheet_mkt',
                _mkt='LIVE',
                _asof=self.date,
                _mktdefs=mkt_defs,
                _calprov=mx_cals,
                _mtgdts=mtg_dts,
                _crvdbsvr="SQLP1-A",
                _crvdbnm="Curves",
                _mdssvr="localhost",
                _mdsport=60000,
                _snapdbsvr="SQLP1-B",
                _snapdbnm="Snap",
            )
    
    @property
    def mkt_snap(self):
        return bh.bhCurveMarketSnap(
            _handle='crv_mkt_snap',
            _market=self.sheet_mkt,
            _curves=self.curves
        )
    
    @property
    def mkt_snap_db(self):
        return bh.bhCurveMarketSnapAsDb(
            _handle='crv_mkt_snap_db',
            _marketsnap=self.mkt_snap
        )
    
    @property
    def mkt_snap_db_curves(self):
        return bh.bhDbLookUp(
            _rsid=self.mkt_snap_db,
            _lookupvalues="LIVE",
            _lookupcols="name",
            _returncols="curves"
        )
    
    @property
    def db_curves(self):
        return self.mkt_snap_db_curves
    
    
class COBCurvesRemote:
    def __init__(self, date=None, curves=None, offset=None):
        self.date = date or pd.Timestamp.today().floor('d') - BDay()
        if offset:
            self.date = pd.Timestamp.today().floor('d') - parse_offset(offset)
        self.curves = curves
        
        self.db_mkt = bh.bhCreateCurvesDbMarket(
            _handle=f'crv_db_mkt_{self.date}',
            _mktname="COB",
            _asofdate=self.date,
            _datadate=self.date,
            _mktdefs=mkt_defs,
            _cals=mx_cals,
            _curvesserver="SQLP1-A",
            _curvesdb="Curves",
            _minextraptnr='80y'
        )
        
        self.mkt_snap_cob = bh.bhMarketSnap(
            _handle=f'mkt_snap_cob_{self.date:%Y%m%d}',
            _market=self.db_mkt,
            _curves=self.curves
        )
        
        self.db_curves = self.mkt_snap_cob

def get_sw_rate_from_ccy(
    ccy,
    tenor,
    db_curves,
    curves_df
):
    try:
        curves_df = curves_df.set_axis(curves_df.index.str.lower())
        disc_ = bh.bhDbLookUp(
            _rsid=db_curves, 
            _lookupvalues=curves_df.loc[ccy.lower(), 'disc'],
            _lookupcols="name",
            _returncols="curve",
            _matchtype=0
        )

        proj_ = bh.bhDbLookUp(
            _rsid=db_curves, 
            _lookupvalues=curves_df.loc[ccy.lower(), 'std'],
            _lookupcols="name",
            _returncols="curve",
            _matchtype=0
        )

        rate = bh.bhRate2(
            _crv=disc_,
            _mtyortenor=tenor,
            _type='sw',
            _name=curves_df.loc[ccy.lower(), 'std'],
            _mktdefs=mkt_defs,
            _calprov=mx_cals,
            _leavemtyflag=1,
            _mul=100,
            _projcurve=proj_,
            _mtgdates=mtg_dts
        )
    except Exception as e:
        print(e, ccy, tenor)
        return np.nan
    return rate

def get_sw_rate_from_generator(
    generator, 
    tenor, 
    curves,
    collateral='LCH'
):
    rate = bh.bhFiRate(
        _efforfwd=tenor.lower().split('x')[0],
        _mtyortenor=tenor.lower().split('x')[1],
        _generatorname=generator,
        _curves=curves.db_curves,
        _collateral=collateral,
        _generators=gens
    )
    return rate


def get_sw_rate_from_params(
    params: [list, pd.DataFrame], 
    curves,
    curves_df
):
    
    '''
    param can be a list of dictionaries with keys 'ccy' and tenor or a dataframe with columns ccy and tenor 
    '''
    
    if isinstance(params, list):
        params = pd.DataFrame(params)
    db_curves = curves.db_curves
    rates = [get_sw_rate_from_ccy(row['ccy'], row['tenor'], db_curves=db_curves, curves_df=curves_df) for index, row in params.iterrows()]
    return rates

def parse_params_df(data):
    df = data

    params = []
    for index, row in df.iterrows():
        name, ccy = row['name'], row['ccy']
        row = row.dropna()
        row_sub = row[2:]
        grouper = [l[0] for l in row_sub.index.str.findall('\d+')]
        g = row_sub.groupby(grouper, sort=False)
        row_params = []
        for index, group in g:
            try:
                param = {'name': name, 'ccy': ccy, 'tenor': group[f'leg{index}'], 'wgt': group[f'wgt{index}'], 'group': index}
                row_params.append(param)
            except Exception as e:
                print(e, group, index, '\n')

        params.extend(row_params)
    return params

def update_flies_data(data, live_curves, cob_curves, curves_df):
    params = parse_params_df(data)
    flat_data = pd.DataFrame(params)
    flat_data = flat_data.assign(
        rate=get_sw_rate_from_params(params, live_curves, curves_df),
        cob=get_sw_rate_from_params(params, cob_curves, curves_df)
    )
    
    new_rows = []
    g = flat_data.groupby('name', sort=False)
    for index, group in g:
        group['group'] = group['group'].astype(int)
        group = group.sort_values('group')
        new_row = {'name': group.iloc[0]['name'], 'ccy': group.iloc[0]['ccy']}

        for _, row in group.iterrows():
            new_row[f"wgt{row['group']}"] = row['wgt']
            new_row[f"leg{row['group']}"] = row['tenor']

        for _, row in group.iterrows():
            new_row[f"rate{row['group']}"] = row['rate']
        new_row['live'] = group['wgt'] @ group['rate']

        for _, row in group.iterrows():    
            new_row[f"change{row['group']}"] = row['rate'] - row['cob']
        new_row['change'] = group['wgt'] @ (row['rate'] - group['cob'])
        new_rows.append(new_row)
    new_data = pd.DataFrame(new_rows)
    
    cols = [
        'name', 
        'ccy', 
        *chain.from_iterable([
            *zip(
                new_data.columns[new_data.columns.str.contains('wgt')], 
                new_data.columns[new_data.columns.str.contains('leg')]
            )
        ]),

        *new_data.columns[new_data.columns.str.contains('rate')],
        'live',
        *new_data.columns[new_data.columns.str.contains('change') & new_data.columns.str.contains('\d+')],
        'change'
    ]
    
    return new_data[cols]

live_curves = LiveCurvesLocal(curves=curves_df.melt()['value'].to_list())
cob_curves = COBCurvesRemote(curves=curves_df.melt()['value'].to_list())

import threading
import time
from functools import partial


class RepeatedTimer:
    def __init__(self, function, interval):
        self._timer = None
        self.interval = interval
        self.function = function
        self.is_running = False

    def _run(self, *args, **kwargs):
        self.is_running = False
        self.start()
        self.function(*args, **kwargs)

    def start(self, *args, **kwargs):
        if not self.is_running:
            if hasattr(self, 'next_call'):
                self.next_call += self.interval
            else:
                self.next_call = time.time()
            self._timer = threading.Timer(self.next_call - time.time(), partial(self._run, *args, **kwargs))
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


class EditableGrid(View):
    def __init__(self, data=None, curves=None, **kwargs):
        self.dg = self.make_dg(data)
        self._curves = curves
        self.live_curves = LiveCurvesLocal(curves=curves)
        self.cob_curves = COBCurvesRemote(curves=curves)
        super().__init__(**kwargs)
    
    def update_dg(self, data=None):
        self.dg = self.make_dg(data)
    
    def make_widgets(self, **kwargs):
        add_btn = v.Btn(
                fab=True,
                # small=True,
                x_small=True,
                plain=True,
                class_='ma-2',
                children=[v.Icon(children=['mdi-plus'])]
        )

        rmv_btn = v.Btn(
                fab=True,
                # small=True,
                x_small=True,
                plain=True,
                class_='ma-2',
                children=[v.Icon(children=['mdi-minus'])]
        )


        above_btn = v.Btn(
                fab=True,
                # small=True,
                x_small=True,
                plain=True,
                class_='ma-2',
                children=[v.Icon(children=['mdi-arrow-up-bold'])]
        )

        below_btn = v.Btn(
                fab=True,
                # small=True,
                x_small=True,
                plain=True,
                class_='ma-2',
                children=[v.Icon(children=['mdi-arrow-down-bold'])]
        )
        
        file_tf = v.TextField(
            label='file name',
            v_model=None,
            # prepend_icon='mdi-content-save',
            outlined=True,
            dense=True
        )

        save_btn = v.Btn(
            fab=True,
            # small=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-content-save'])]
        )

        load_btn = v.Btn(
            fab=True,
            # small=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-file-upload'])]
        )
        
        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-sync'])]
        )
        
        timer_tf = v.TextField(
            v_model=45,
            label='time',
            clearable=True,
            dense=True,
            outlined=True,
            type="number",
            class_="mr-0"

        )
        
        change_tf = v.TextField(
            v_model='1d',
            label='change',
            clearable=True,
            dense=True,
            outlined=True,
            class_="mr-0"

        )
        
        timer_btn = v.Btn(
            fab=True,
            # dark=True,
            x_small=True,
            plain=True,
            class_='mt-2 mb-2 mr-2 ml-0',
            children=[v.Icon(children=['mdi-timer'])]
        )
    
        self.add_btn = add_btn
        self.rmv_btn = rmv_btn
        self.above_btn = above_btn
        self.below_btn = below_btn
        self.file_tf = file_tf
        self.save_btn = save_btn
        self.load_btn = load_btn
        self.refresh_btn = refresh_btn
        self.timer_tf = timer_tf
        self.timer_btn = timer_btn
        self.change_tf = change_tf
        self.expr_dg = EditableGrid2()
        
        self.btn_box = w.HBox(
            [
                add_btn,
                rmv_btn,
                above_btn,
                below_btn
            ]
        )
        
    def make_view(self, **kwargs):
        self.param_box = v.Row(
            children=[
                v.Col(
                    cols=3,
                    children=[self.btn_box],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=4,
                    children=[w.HBox(children=[self.file_tf, self.load_btn, self.save_btn])],
                    class_="my-0 py-0"
                ),
                # v.Col(
                #     cols=2,
                #     children=[],
                #     class_="my-0 py-0"
                # ),
                # v.Col(
                #     cols=3,
                #     children=[self.refresh_btn],
                #     class_="my-0 py-0"
                # ),
                v.Col(
                    cols=1.0,
                    children=[self.timer_tf],
                    class_="my-0 py-0 mr-0 pr-0"
                ),
                v.Col(
                    cols=2,
                    children=[self.timer_btn, self.refresh_btn],
                    class_="my-0 py-0"
                ),
                
                v.Col(
                    cols=1.0,
                    children=[self.change_tf],
                    class_="my-0 py-0 mr-0 pr-0"
                ),
            ]
        )
        
        self.view = w.VBox([self.param_box, self.dg, self.expr_dg.view])
    
    input_cols = [
        'name',
        'ccy',
        'wgt1',
        'leg1',
        'wgt2',
        'leg2',
        'wgt3',
        'leg3',
    ]

    output_cols = [
        'rate1',
        'rate2',
        'rate3',
        'live',
        'change1',
        'change2',
        'change3',
        'change'
    ]
    
    def make_dg(self, data=None, **kwargs):
        input_cols = self.input_cols
        output_cols = self.output_cols

        wgt_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            text_color=Expr('"red" if cell.value < 0 else default_value'),
        )

        float_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.3f')"),
            # text_color=Expr('"red" if cell.value < 0 else default_value'),
        )

        renderers = {
            'wgt1': wgt_renderer ,
            'wgt2': wgt_renderer,
            'wgt3': wgt_renderer,
            'rate1': float_renderer,
            'rate2': float_renderer,
            'rate3': float_renderer,
            'live': float_renderer,
            'change1': float_renderer,
            'change2': float_renderer,
            'change3': float_renderer,
            'change': float_renderer
        }
        
        data = pd.DataFrame([self.blank_data], columns=[*input_cols, *output_cols]).reindex([0]) if data is None else data
        dg = DataGrid(
            data, 
            selection_mode='cell', 
            editable=True,
            base_column_size=75,
            renderers=renderers,
            column_widths={
                'name': 150
            }
        )

        return dg
    
    def link(self, **kwargs):
        
        self.add_btn.on_event(
            'click',
            partial(on_click_add, dg=self.dg, blank_data=[self.blank_data])
        )
        self.rmv_btn.on_event(
            'click',
            partial(on_click_remove, dg=self.dg)
        )
        self.above_btn.on_event(
            'click',
            partial(on_click_add_above, dg=self.dg, blank_data=[self.blank_data])
        )
        self.below_btn.on_event(
            'click',
            partial(on_click_add_below, dg=self.dg, blank_data=[self.blank_data])
        )
        
        self.load_btn.on_event(
            'click',
            partial(on_click_upload, self=self)
        )
        
        self.save_btn.on_event(
            'click',
            partial(on_click_save, self=self)
        )
        self.refresh_btn.on_event(
            'click',
            partial(on_click_refresh, self=self)
        )
        self.timer_btn.on_event(
            'click',
            partial(on_click_change_timer, self=self)
        )
        
        self.change_tf.on_event(
            'change',
            partial(on_change_update_cob_curves, self=self)
        )
        super().link(**kwargs)
    
    @property
    def blank_data(self):
        _blank_data = [
            "",
            "",
            np.nan,
            "",
            np.nan,
            "",
            np.nan,
            "",
            np.nan,
            np.nan,
            np.nan,
            np.nan,
            np.nan,
            np.nan,
            np.nan,
            np.nan,
        ]
        return _blank_data
    
    def update_cob_curves(self):
        self.cob_curves = COBCurvesRemote(curves=self._curves, offset=self.change_tf.v_model)
        
        
    def upload_file(self):
        file_name = radar_path / 'save' / f'{self.file_tf.v_model}_flies.csv'
        data = pd.read_csv(file_name)
        current_data = self.dg.data
        self.dg.data = pd.concat([current_data, data])[len(current_data):].reset_index(drop=True)
        
        try:
            file_name = radar_path / 'save' / f'{self.file_tf.v_model}_expr.csv'
            data = pd.read_csv(file_name)
            expr_data = self.expr_dg.dg.data
            self.expr_dg.dg.data = pd.concat([expr_data, data])[len(expr_data):].reset_index(drop=True)
        except FileNotFoundError:
            pass
        
    def save_to_file(self):
        file_name = radar_path / 'save' / f'{self.file_tf.v_model}_flies.csv'
        current_data = self.dg.data
        current_data[self.input_cols].to_csv(file_name, index=False)
        
        file_name = radar_path / 'save' / f'{self.file_tf.v_model}_expr.csv'
        expr_data = self.expr_dg.dg.data
        expr_data[self.expr_dg.input_cols].to_csv(file_name, index=False)
        
        
        
    def refresh_data(self):
        live_curves = self.live_curves
        cob_curves = self.cob_curves
        new_data = update_flies_data(
            data=self.dg.data[self.input_cols], 
            live_curves=live_curves, 
            cob_curves=cob_curves, 
            curves_df=curves_df
        )
        
        self.dg.data = new_data
        
        self.calc_expressions()
        print('updated')
        
    def calc_expressions(self):
        expr_data = self.expr_dg.dg.data
        expr_data = expr_data.loc[expr_data['expression'] != ""]
        if not expr_data.empty:
            live = bh.bhTsCalcExpression(
                _resultnames=range(len(expr_data)),  # if omitted it doesn't work with multiple expressions at once
                _expressions=expr_data['expression'].to_list(),
                _varnamesorhndl=self.dg.data['name'].to_list(),
                _varvalues=self.dg.data['live'].to_list()
            ).squeeze(axis=1)

            change = bh.bhTsCalcExpression(
                _resultnames=range(len(expr_data)),
                _expressions=expr_data['expression'].to_list(),
                _varnamesorhndl=self.dg.data['name'].to_list(),
                _varvalues=self.dg.data['change'].to_list()
            ).squeeze(axis=1)

            expr_data = expr_data.assign(
                live=live,
                change=change
            )

            self.expr_dg.dg.data = expr_data
    
    def start_stop_timer(self):
        sec = int(self.timer_tf.v_model)
        try:
            rt = self.rt
            if not rt.is_running:
                rt.interval = sec
        except AttributeError:
            self.rt = RepeatedTimer(self.refresh_data, sec)
            rt = self.rt
        
        if rt.is_running:
            self.timer_btn.color = None
            rt.stop()
        else:
            self.timer_btn.color = 'success'
            rt.start()
            

class EditableGrid2(View):
    def __init__(self, data=None, **kwargs):
        self.dg = self.make_dg(data)
        super().__init__(**kwargs)

    def update_dg(self, data=None):
        self.dg = self.make_dg(data)

    def make_widgets(self, **kwargs):
        add_btn = v.Btn(
            fab=True,
            # small=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-plus'])]
        )

        rmv_btn = v.Btn(
            fab=True,
            # small=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-minus'])]
        )

        above_btn = v.Btn(
            fab=True,
            # small=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-arrow-up-bold'])]
        )

        below_btn = v.Btn(
            fab=True,
            # small=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-arrow-down-bold'])]
        )

        self.add_btn = add_btn
        self.rmv_btn = rmv_btn
        self.above_btn = above_btn
        self.below_btn = below_btn
        self.btn_box = w.HBox(
            [
                add_btn,
                rmv_btn,
                above_btn,
                below_btn
            ]
        )

    def make_view(self, **kwargs):
        self.param_box = v.Row(
            children=[
                v.Col(
                    cols=3,
                    children=[self.btn_box],
                    class_="my-0 py-0"
                ),
            ]
        )

        self.view = w.VBox([self.param_box, self.dg])

    input_cols = [
        'expression',
        'sign',
        'target',
    ]

    output_cols = [
        'live',
        'change',
    ]

    def make_dg(self, data=None, **kwargs):
        input_cols = self.input_cols
        output_cols = self.output_cols

        change_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.3f')"),
            text_color=Expr('"red" if cell.value < 0 else default_value'),
        )

        live_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.3f')"),
            # text_color=Expr('"red" if cell.value < 0 else default_value'),
            background_color=VegaExpr(
                "(cell.value >= cell.metadata.data['target'] && cell.metadata.data['sign'] == '+') || \
                (cell.value <= cell.metadata.data['target'] && cell.metadata.data['sign'] == '-')  ? 'red' : default_value"
            )
        )

        float_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.3f')"),
            # text_color=Expr('"red" if cell.value < 0 else default_value'),
        )

        renderers = {
            'target': float_renderer,
            'live': live_renderer,
            'change': change_renderer,
        }

        data = pd.DataFrame([self.blank_data], columns=[*input_cols, *output_cols]).reindex(
            [0]) if data is None else data
        dg = DataGrid(
            data,
            selection_mode='cell',
            editable=True,
            base_column_size=75,
            renderers=renderers,
            column_widths={
                'expression': 350,
                'sign': 60
            }
        )

        return dg

    def link(self, **kwargs):
        self.add_btn.on_event(
            'click',
            partial(on_click_add, dg=self.dg, blank_data=[self.blank_data])
        )
        self.rmv_btn.on_event(
            'click',
            partial(on_click_remove, dg=self.dg)
        )
        self.above_btn.on_event(
            'click',
            partial(on_click_add_above, dg=self.dg, blank_data=[self.blank_data])
        )
        self.below_btn.on_event(
            'click',
            partial(on_click_add_below, dg=self.dg, blank_data=[self.blank_data])
        )

    @property
    def blank_data(self):
        _blank_data = [
            "",
            "+",
            np.nan,
            np.nan,
            np.nan,
        ]
        return _blank_data

def remove_row(dg):
    current_data = dg.data
    selections = dg.selections
    if not selections:
        new_data = current_data[:-1]
    else:
        rows = []
        for sel in selections:
            sel_rows = [*range(sel['r1'], sel['r2'] + 1)]
            rows.extend(sel_rows)
        to_drop = pd.Series(rows).drop_duplicates().to_list()
        new_data = current_data.drop(current_data.index[to_drop]).reset_index(drop=True)
    dg.data = new_data
    

def add_row(dg, blank_data=None):
    current_data = dg.data
    new_row = pd.DataFrame(
        data=blank_data,
        columns=current_data.columns,
        index=[0]
    )

    new_data = pd.concat([current_data, new_row]).reset_index(drop=True)
    dg.data = new_data
    

def add_above(dg, blank_data=None):
    current_data = dg.data
    selections = dg.selections
    if not selections:
        return
    
    new_row = pd.DataFrame(
        data=blank_data,
        columns=current_data.columns,
        index=[0]
    )
    
    sel = selections[0]['r1']
    above = current_data.iloc[:sel]
    below = current_data.iloc[sel:]
    new_data = pd.concat([above, new_row, below]).reset_index(drop=True)
    dg.data = new_data
    
    
def add_below(dg, blank_data=None):
    current_data = dg.data
    selections = dg.selections
    if not selections:
        return
    
    new_row = pd.DataFrame(
        data=blank_data,
        columns=current_data.columns,
        index=[0]
    )
    
    sel = selections[0]['r1']
    above = current_data.iloc[:sel+1]
    below = current_data.iloc[sel+1:]
    new_data = pd.concat([above, new_row, below]).reset_index(drop=True)
    dg.data = new_data

def on_click_add(widgent, event, payload, dg, blank_data=None):
    add_row(dg, blank_data=blank_data)

def on_click_remove(widgent, event, payload, dg):
    remove_row(dg)
    
def on_click_add_above(widgent, event, payload, dg, blank_data=None):
    add_above(dg, blank_data=blank_data)

def on_click_add_below(widgent, event, payload, dg, blank_data=None):
    add_below(dg, blank_data=blank_data)
    
def on_click_upload(widget, event, payload, self):
    self.upload_file()
    
def on_click_save(widget, event, payload, self):
    self.save_to_file()
    
def on_click_refresh(widget, event, payload, self):
    self.refresh_data()
    
def on_click_change_timer(widget, event, payload, self):
    self.start_stop_timer()
    
def on_change_update_cob_curves(widget, event, payload, self):
    self.update_cob_curves()
    

class Radar(View):
    
    def __init__(self, curves=None, **kwargs):
        self._curves = curves
        super().__init__(**kwargs)

    def make_widgets(self):
        editable_grid = EditableGrid(curves=self._curves)
        tabs_bars = v.Tabs(
            # dark=True,
            v_model=None,
            children=[
                v.Tab(children=['flies']), 
                v.Tab(children=['watchlist'])
            ]
        )

        t1 = w.VBox([editable_grid.param_box, editable_grid.dg])
        t2 = editable_grid.expr_dg.view

        tabs_items = v.TabsItems(
            v_model=None,
            # dark=True,
            children=[
                v.TabItem(children=[t1]),
                v.TabItem(children=[t2])
            ]
        )
        
        self.editable_grid = editable_grid
        self.tabs_bars = tabs_bars
        self.t1 = t1
        self.t2 = t2
        self.tabs_items = tabs_items

    def make_view(self):
        self.view = v.Card(
            children=[
                self.tabs_bars,
                self.tabs_items
            ]
        )

    def link(self):
        w.jslink((self.tabs_bars, 'v_model'), (self.tabs_items, 'v_model'))
        super().link()


self = Radar(curves=curves_df.melt()['value'].to_list())
self

self.editable_grid.cob_curves.off

NameError: name 'pd' is not defined